In [ ]:
Antonio Juarez Alonso
Oliver Azael Vazquez Juarez
Juan Carlos Jimenez Gutierrez
Carlos Enrique Perez Perez

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import matplotlib.pyplot as plt

In [ ]:
cifar = tf.keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar.load_data(label_mode='coarse')
x_train, x_test = x_train / 255.0, x_test / 255.0

# Agrega una dimension de canales
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(64, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(256, activation='relu')
    self.d2 = Dense(20, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Crea una instancia del modelo
model = MyModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 30

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Perdida: {}, Exactitud: {}, Perdida de prueba: {}, Exactitud de prueba: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reinicia las metricas para el siguiente epoch.
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Perdida: 2.276411533355713, Exactitud: 30.540000915527344, Perdida de prueba: 2.0718443393707275, Exactitud de prueba: 36.29999923706055
Epoch 2, Perdida: 1.9124869108200073, Exactitud: 41.2339973449707, Perdida de prueba: 1.994435429573059, Exactitud de prueba: 38.61000061035156
Epoch 3, Perdida: 1.68035888671875, Exactitud: 47.97999954223633, Perdida de prueba: 1.9545239210128784, Exactitud de prueba: 40.93000030517578
Epoch 4, Perdida: 1.427721619606018, Exactitud: 55.64400100708008, Perdida de prueba: 2.0374763011932373, Exactitud de prueba: 40.66999816894531
Epoch 5, Perdida: 1.1569005250930786, Exactitud: 63.83799743652344, Perdida de prueba: 2.1969237327575684, Exactitud de prueba: 40.7599983215332
Epoch 6, Perdida: 0.8959044218063354, Exactitud: 72.07400512695312, Perdida de prueba: 2.3882436752319336, Exactitud de prueba: 40.2599983215332
Epoch 7, Perdida: 0.673579216003418, Exactitud: 79.14399719238281, Perdida de prueba: 2.730087995529175, Exactitud de prueba: 39.02